In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install transformers

In [33]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments,Trainer

In [34]:
TRAINING_PATH = "train.csv"
TEST_PATH = "test.csv"

### Importing and EDA

In [35]:
df = pd.read_csv(TRAINING_PATH)
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [36]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [37]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' +df.anchor

In [38]:
df.input.head()

,input
0,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,TEXT1: A47; TEXT2: forest region; ANC1: abatement


### Tokenization

In [39]:
ds = Dataset.from_pandas(df)

In [40]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [41]:
model_nm = 'microsoft/deberta-v3-small'

In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_nm)

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [43]:
def tok_func(x): return tokenizer(x["input"])

In [44]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [45]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [46]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

### Testing and validation

In [58]:
test_df = pd.read_csv(TEST_PATH)
test_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [59]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [60]:
test_df['input'] = 'TEXT1: ' + test_df.context + '; TEXT2: ' + test_df.target + '; ANC1: ' +test_df.anchor
test_ds = Dataset.from_pandas(test_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

###Metrics

In [50]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

### Training


In [51]:
bs = 128
epochs = 4
lr=8e-5


In [52]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True, evaluation_strategy = "epoch",
                         per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2, num_train_epochs=epochs,
                         weight_decay=0.01, report_to='none')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [53]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                tokenizer=tokenizer, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-53-b36f4e250fc6>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [54]:
trainer.train();

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.028754,0.799202
2,No log,0.021999,0.823296
3,0.033000,0.021433,0.833860
4,0.033000,0.022446,0.833731


In [61]:
preds = trainer.predict(test_ds).predictions.astype(float)
preds

array([[ 5.42480469e-01],
       [ 6.42089844e-01],
       [ 5.67871094e-01],
       [ 3.20556641e-01],
       [-8.72039795e-03],
       [ 4.85351562e-01],
       [ 5.18554688e-01],
       [-4.36096191e-02],
       [ 2.51708984e-01],
       [ 1.10058594e+00],
       [ 2.87841797e-01],
       [ 2.46826172e-01],
       [ 7.65625000e-01],
       [ 9.55566406e-01],
       [ 7.19726562e-01],
       [ 3.95263672e-01],
       [ 2.94189453e-01],
       [-5.64193726e-03],
       [ 6.19628906e-01],
       [ 3.74755859e-01],
       [ 4.09912109e-01],
       [ 2.05322266e-01],
       [ 3.70178223e-02],
       [ 2.79296875e-01],
       [ 6.07421875e-01],
       [-9.23156738e-04],
       [-1.70612335e-03],
       [-3.28063965e-02],
       [-4.55932617e-02],
       [ 6.67968750e-01],
       [ 3.37402344e-01],
       [ 6.27441406e-02],
       [ 7.08984375e-01],
       [ 5.28808594e-01],
       [ 4.29199219e-01],
       [ 2.18383789e-01]])

In [62]:
preds = np.clip(preds, 0, 1)

In [63]:
preds

array([[0.54248047],
       [0.64208984],
       [0.56787109],
       [0.32055664],
       [0.        ],
       [0.48535156],
       [0.51855469],
       [0.        ],
       [0.25170898],
       [1.        ],
       [0.2878418 ],
       [0.24682617],
       [0.765625  ],
       [0.95556641],
       [0.71972656],
       [0.39526367],
       [0.29418945],
       [0.        ],
       [0.61962891],
       [0.37475586],
       [0.40991211],
       [0.20532227],
       [0.03701782],
       [0.27929688],
       [0.60742188],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.66796875],
       [0.33740234],
       [0.06274414],
       [0.70898438],
       [0.52880859],
       [0.42919922],
       [0.21838379]])